## Train a neural network to classify regions in a 2D space depending on their distance to select points (classification)

In [ ]:
# Imports

import numpy as np

import matplotlib.pyplot as plt

import copy as copy

from neuralnetwork.neural_network_module import NeuralNetwork, Layer
from neuralnetwork.neural_network_module import train_nn_classification, one_hot_to_predictions, confusion_matrix

np.random.seed(101) # set random seed

plt.rcParams.update({'font.size': 12}) # plot parameters

### Functions to label points in the (x,y) plane according to their L1 or L2 norm distance from select points

In [ ]:
# Functions to label points in the (x,y) plane

# label points by their L1 norm distance along the x-axis to specified points
def grid_X_by_L1(X, points, distances):
    yy = np.zeros((X.shape[0],1))
    for kd, distance in enumerate(distances):
        for kp in points:
            idx = np.where(abs(X[:,0] - kp) <= distance)
            yy[idx,0] = kd+1
    return yy

# label points by their L1 norm distance along the y-axis to specified points
def grid_Y_by_L1(X, points, distances):
    yy = np.zeros((X.shape[0],1))
    for kd, distance in enumerate(distances):
        for kp in points:
            idx = np.where(abs(X[:,1] - kp) <= distance)
            yy[idx,0] = kd+1
    return yy

# label points by their L1 norm distance to specified points
def grid_by_L1(X, points, distances):
    yy = np.zeros((X.shape[0],1))
    for kd, distance in enumerate(distances):
        for kx in points:
            for ky in points:
                # idx = np.where((abs(X[:,0] - kx) <= distance) | (abs(X[:,1] - ky) <= distance))
                # idx = np.where((abs(X[:,0] - kx) <= distance) & (abs(X[:,1] - ky) <= distance))
                idx = np.where(abs(X[:,0] - kx) + abs(X[:,1] - ky) <= distance)
                yy[idx] = kd+1
    return yy

# label points by their L2 norm distance to specified points
def grid_by_L2(X, points, distances):
    yy = np.zeros((X.shape[0],1))
    for kd, distance in enumerate(distances):
        for kx in points:
            for ky in points:
                idx = np.where(((X[:,0] - kx)**2 + (X[:,1] - ky)**2 <= distance**2))
                yy[idx] = kd+1
    return yy

### Create and label data

In [ ]:
# Create data and plot

N = int(1e4) # number of samples

xx = 2*(np.random.rand(N,2) - 0.5)

yy = np.zeros((N,4), dtype=int) # labels
yy[:,[0]] = grid_X_by_L1(xx, np.arange(-1,1.5,0.5), [0.1, 0.05])
yy[:,[1]] = grid_Y_by_L1(xx, np.arange(-1,1.5,0.5), [0.1, 0.05])
yy[:,[2]] = grid_by_L1(xx, np.arange(-1,1.5,0.5), [0.2, 0.1])
yy[:,[3]] = grid_by_L2(xx, np.arange(-1,1.5,0.5), [0.2, 0.1])

plt.figure(figsize=(24,6))
plt.subplot(1,4,1)
plt.scatter(xx[:,0], xx[:,1], c=yy[:,0], cmap="jet", s=2)
plt.title("Labeled by L1 norm along x axis")
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1,4,2)
plt.scatter(xx[:,0], xx[:,1], c=yy[:,1], cmap="jet", s=2)
plt.title("Labeled by L1 norm along y axis")
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1,4,3)
plt.scatter(xx[:,0], xx[:,1], c=yy[:,2], cmap="jet", s=2)
plt.title("Labeled by L1 norm distance")
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1,4,4)
plt.scatter(xx[:,0], xx[:,1], c=yy[:,3], cmap="jet", s=2)
plt.title("Labeled by L2 norm distance")
plt.xlabel("x")
plt.ylabel("y")
plt.show()


### Split data into training and test sets

In [ ]:
# Split data into train and test sets

train_test_split = 0.8

# split into train and test sets
idx_train = range(0,int(train_test_split*xx.shape[0])) # training set indexes
X_train = xx[idx_train,:]
Y_train = yy[idx_train,:]

idx_test = range(int(train_test_split*xx.shape[0]),xx.shape[0]) # test set indexes
X_test = xx[idx_test,:]
Y_test = yy[idx_test,:]

### Select label on which to train NN

In [ ]:
# Select norm data for training neural network, and plotting results

idx_norm = 2

### Create and train neural network and analyze its performance

In [ ]:
# Create and train the neural network

nn = NeuralNetwork()

layer_list = []
# layer_list.append(Layer())

nn._add_layers([Layer(2, None)])
nn._add_layers([Layer(64, "relu")])
nn._add_layers([Layer(64, "relu")])
nn._add_layers([Layer(64, "relu")])
nn._add_layers([Layer(64, "relu")])
# nn._add_layers([Layer(16, "relu")])
# nn._add_layers([Layer(16, "relu")])
# nn._add_layers([Layer(16, "relu")])
# nn._add_layers([Layer(16, "relu")])
nn._add_layers([Layer(64, "linear")])
nn._add_layers([Layer(3, "softmax")])

NN_list, train_accuracy, test_accuracy, epochs = train_nn_classification(nn, X_train, Y_train[:,[idx_norm]], 0.020, 5000, 500, X_test, Y_test[:,[idx_norm]])


In [ ]:
# Plot accuracy of NN across training epochs

plt.figure(figsize=(12,6))
plt.plot(epochs, train_accuracy, 'b.', label="Train accuracy")
if len(test_accuracy) == len(epochs):
    plt.plot(epochs, test_accuracy, 'g.', label="Test accuracy")
plt.xticks(epochs)
plt.xlabel("Iterations")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
# Plot training data and NN predictions

# NN predictions on training and test sets
Y_pred_train = one_hot_to_predictions(nn._compute_output(X_train))
Y_pred_test = one_hot_to_predictions(nn._compute_output(X_test))

#
X_plot = 2*(np.random.rand(int(1e5),2)-0.5) # data for NN predictions across input space
Y_plot = one_hot_to_predictions(nn._compute_output(X_plot)) # NN predictions


plt.figure(figsize=(24,6))
plt.subplot(1,4,1)
plt.scatter(xx[:,0], xx[:,1], c=yy[:,idx_norm], cmap="jet", s=2)
plt.title("Raw Data")
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1,4,2)
plt.scatter(X_train[:,0], X_train[:,1], c=Y_pred_train, cmap="jet", s=2)
plt.title("Predictions on Training Data Set")
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1,4,3)
plt.scatter(X_test[:,0], X_test[:,1], c=Y_pred_test, cmap="jet", s=2)
plt.title("Predictions on Test Data Set")
plt.xlabel("x")
plt.ylabel("y")
plt.subplot(1,4,4)
plt.scatter(X_plot[:,0], X_plot[:,1], c=Y_plot, cmap="jet", s=2)
plt.title("NN Predictions")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
# Plot confusion matrix for NN predictions

CM_train = confusion_matrix(NN_list[-1], X_train, Y_train)
CM_test = confusion_matrix(NN_list[-1], X_test, Y_test)

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(CM_train, cmap="Blues")
plt.xticks([0, 1, 2])
plt.yticks([0, 1, 2])
plt.title("Training CM")
plt.xlabel("Predictions")
plt.ylabel("Values")
plt.subplot(1,2,2)
plt.xticks([0, 1, 2])
plt.yticks([0, 1, 2])
plt.imshow(CM_test, cmap="Blues")
plt.title("Test CM")
plt.xlabel("Predictions")
plt.ylabel("Values")
plt.show()
